In [1]:
import jraph
import jax
import jax.numpy as jnp
import e3nn_jax as e3nn
import numpy as np


import sys
sys.path.append('..')


/Users/ameyad/Documents/spherical-harmonic-net/.venv/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/ameyad/Documents/spherical-harmonic-net/.venv/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [15]:
def f(x, msg: str):
    return (x + 1)

jax.jit(f, static_argnames=("msg",))(1, "hello")


Array(2, dtype=int32, weak_type=True)

In [9]:
num_graphs = 1
num_radii = 4
num_channels = 1
lmax = 4

irreps = e3nn.Irreps(f"{lmax}x0e") + e3nn.s2_irreps(lmax, p_val=1, p_arg=-1)
position_coeffs = e3nn.normal(
    irreps,
    leading_shape=(
        num_graphs,
        num_channels,
        num_radii,
    )
)
print(position_coeffs)
gated_position_coeffs = e3nn.gate(position_coeffs)
print(gated_position_coeffs)

/Users/ameyad/Documents/spherical-harmonic-net/.venv/lib/python3.10/site-packages/e3nn_jax/_src/basic.py:555: UserWarning: e3nn.normal: the key (random seed) is not provided, use the hash of the irreps as key!
  warnings.warn(


4x0e+1x0e+1x1o+1x2e+1x3o+1x4e
[[[[ 0.178965   -0.8160956   0.7827804   0.7025114   0.18371503
     0.30572075 -0.58640146 -1.0551178  -0.18641537 -1.8394815
    -1.6997705   0.12492011 -1.1149572  -0.45623064  2.0284507
     0.62625724  0.19440839 -0.88461185  0.07666336  0.19841796
    -0.41691393  0.95379716  1.4429971   1.163871   -0.16326699
     0.5577532   0.50488    -0.26682705 -0.8007012 ]
   [ 0.24812463 -0.7968878  -0.78703904  0.26025212 -0.2962588
    -1.5016868   1.7713156  -1.3583281   1.5215381   1.5868151
    -0.71454453  0.53214806 -1.0575016   1.0768367   0.39943078
     1.799385    0.30608553  1.5122793   2.3733823   0.01290947
     0.68991685 -1.2643776  -1.054789   -0.802186   -0.89324105
    -0.03816885 -1.6905568   0.5869649   0.42957944]
   [ 0.78078157  0.18377762 -1.5986038  -0.9588251   0.6763224
    -0.16578607 -0.0114271  -0.31424648  0.37393025 -0.14874277
    -0.2690466  -0.9938431  -0.08279011  0.62625223  0.02151301
     0.8201572   0.61040366  1.492888

In [ ]:
log_radius_weights = e3nn.IrrepsArray("0e", jnp.asarray([1.]))
log_angular_coeffs = e3nn.IrrepsArray("0e + 1o", jnp.arange(4).astype(float))
x = e3nn.concatenate([log_radius_weights, log_angular_coeffs], axis=0)
x, x.simplify(), e3nn.sum(x)

In [ ]:
log_radius_weights = e3nn.IrrepsArray("0e", jnp.arange(64).reshape((64, 1)))
log_angular_coeffs = e3nn.IrrepsArray("0e + 1o", jnp.arange(4))
log_angular_coeffs = e3nn.IrrepsArray(log_angular_coeffs.irreps, jnp.tile(log_angular_coeffs.array, (64, 1)))
print(log_angular_coeffs.shape)
e3nn.concatenate([log_radius_weights, log_angular_coeffs], axis=1)

In [ ]:
coeffs = e3nn.IrrepsArray("0e + 1o", jnp.asarray([1.0, 1.0, 2.0, 3.0]))
coeffs_on_grid = e3nn.to_s2grid(coeffs, res_beta=30, res_alpha=59, quadrature="soft").grid_values

scaled_coeffs = coeffs * 2.0
scaled_coeffs_on_grid = e3nn.to_s2grid(scaled_coeffs, res_beta=30, res_alpha=59, quadrature="soft").grid_values

scaled_coeffs_on_grid / coeffs_on_grid

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
import train
import models
import datatypes
import analyses.generate_plots as generate_plots
import input_pipeline_tf
import configs.mace as mace
import analyses.analysis as analysis

In [ ]:
config = mace.get_config()
print(config.train_molecules)
config.train_molecules = (0, 2976)
datasets = input_pipeline_tf.get_datasets(None, config)
for step, _ in enumerate(datasets['train']):
    if step % 50000 == 0:
        print(step)
print(step)
    

In [ ]:
basedir = "../potato_workdirs/extras/sample_complexity"
results = analysis.get_results_as_dataframe(generate_plots.ALL_MODELS, generate_plots.ALL_METRICS, basedir)
results['val'][results['val']['model'] == 'mace'].sort_values(by='max_l')

In [ ]:
results['val'].to_clipboard()

In [ ]:
def eval():
    for x in datasets["val"].take(10).as_numpy_iterator():
        mask = jraph.get_graph_padding_mask(x)
        print(x.n_node[mask].sum())

eval()


In [ ]:
import tensorflow as tf

ds = tf.data.Dataset.range(10)

In [ ]:
for x in ds.take(3).as_numpy_iterator():
    print(x)

In [ ]:
config = mace.get_config()
datasets = input_pipeline_tf.get_raw_qm9_datasets(config)
for x in datasets["val"].take(10).as_numpy_iterator():
    print(x['target_positions'].sum())

In [ ]:
ds = datasets['val'].map(
    input_pipeline_tf._convert_to_graphstuple,
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=True,
)

ds_b = jraph.dynamically_batch(graphs_tuple_iterator=iter(ds),
                                n_node=config.max_n_nodes,
                                n_edge=config.max_n_edges,
                                n_graph=config.max_n_graphs)

In [ ]:
for step, x in enumerate(ds_b):
    mask = jraph.get_graph_padding_mask(x)
    print(x.n_node[mask].sum())
    if step == 10:
        break

In [ ]:
import functools

example_graph = next(ds.as_numpy_iterator())
example_padded_graph = jraph.pad_with_graphs(
    example_graph, n_node=config.max_n_nodes, n_edge=config.max_n_edges, n_graph=config.max_n_graphs
)
padded_graphs_spec = input_pipeline_tf._specs_from_graphs_tuple(example_padded_graph)

# Batch and pad each split separately.
batching_fn = functools.partial(
    jraph.dynamically_batch,
    graphs_tuple_iterator=iter(ds),
    n_node=config.max_n_nodes,
    n_edge=config.max_n_edges,
    n_graph=config.max_n_graphs,
)
ds_tf = tf.data.Dataset.from_generator(
    batching_fn, output_signature=padded_graphs_spec
)
ds_tf = ds_tf.take(100).cache()

In [ ]:
for step, x in enumerate(ds_tf.as_numpy_iterator()):
    mask = jraph.get_graph_padding_mask(x)
    print(x.n_node[mask].sum())
    if step == 10:
        break

In [ ]:
# Mimic what we do in train.py.
config = mace.get_config()

config.max_n_graphs = 32
rng = jax.random.PRNGKey(0)
rng, dataset_rng = jax.random.split(rng)

# Obtain graphs.
datasets = input_pipeline_tf.get_datasets(dataset_rng, config)
train_iter = datasets["train"].as_numpy_iterator()
init_graphs = next(train_iter)

# Set up dummy variables to obtain the structure.
rng, init_rng = jax.random.split(rng)
net = train.create_model(config, run_in_evaluation_mode=False)
params = jax.jit(net.init)(init_rng, init_graphs)

#ds = ds.shuffle(buffer_size=5, reshuffle_each_iteration=False)

In [ ]:
for x in datasets["train_eval_final"].as_numpy_iterator():
    mask = jraph.get_graph_padding_mask(x)
    print(x.n_node[mask].sum())

In [ ]:
import tensorflow as tf
ds = tf.data.Dataset.from_tensor_slices([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
ds = ds.shuffle(buffer_size=5, reshuffle_each_iteration=False)

In [ ]:
for x in ds.as_numpy_iterator():
    print(x)

In [ ]:
print("padding")
print("nodes", config.max_n_nodes)
print("edges", config.max_n_edges)
print("graphs", config.max_n_graphs)

In [ ]:
count = 0
for step, graphs in enumerate(datasets["train"].as_numpy_iterator()):
    if step % 1000 == 1:
        print(step, count / step)
    if step == 10000:
        break

    graphs = jax.tree_map(jnp.array, graphs)
    graphs = datatypes.Fragments.from_graphstuple(graphs)
    count += jraph.get_graph_padding_mask(graphs).sum()

print(count / step)


In [ ]:
graphs = jraph.GraphsTuple(
    nodes=jnp.array([[1, 2, 3], [1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    edges=jnp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    globals=jnp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    n_node=jnp.array([2, 1, 1]),
    n_edge=jnp.array([1, 1, 1]),
    senders=jnp.array([0, 0, 0]),
    receivers=jnp.array([0, 0, 0]),
)

In [ ]:
num_graphs = 4
rng, *species_rngs = jax.random.split(jax.random.PRNGKey(0), num_graphs + 1)

target_species_probs = jnp.ones((num_graphs, models.NUM_ELEMENTS)) / models.NUM_ELEMENTS
target_species = jax.vmap(lambda key, p: jax.random.choice(
    key, models.NUM_ELEMENTS, p=p))((species_rngs,), target_species_probs)


In [ ]:
e3nn.IrrepsArray("10x0e", jnp.arange(10))

In [ ]:

myhost

In [ ]:
# Get results.
results = analysis.get_results_as_dataframe(generate_plots.ALL_MODELS, generate_plots.ALL_METRICS, "../potato_workdirs/v4/")

In [ ]:
results['val'].sort_values('total_loss')

In [ ]:
import re
seed, start, end = [int(val) for val in re.findall(r'\d+', "fragments_seed01_from130944_to133920")]
start, end

In [ ]:
def segment_sample(probabilities, segment_ids, num_segments, rng):
    """Sample from a categorical distribution across each segment.
    Args:
        segment_ids: A 1D array of segment ids.
        probs: A 1D array of probabilities.
    Returns:
        A 1D array of samples.
    """
    def sample_for_segment(rng, i):
        return jax.random.choice(rng, node_indices, p=jnp.where(i == segment_ids, probabilities, 0.))
    
    node_indices = jnp.arange(len(segment_ids))
    rngs = jax.random.split(rng, num_segments)
    return jax.vmap(sample_for_segment)(rngs, jnp.arange(num_segments))

In [ ]:
focus_logits = graphs.nodes.sum(axis=1)
probs = jraph.partition_softmax(focus_logits, graphs.n_node)
print(probs)
for seed in range(100):
    print(segment_sample(probs, jnp.asarray([0, 0, 1, 2]), 3, jax.random.PRNGKey(seed)))

In [ ]:
nodes = jnp.asarray([2, 3, 1, 4])
segment_max = e3nn.scatter_max(nodes, nel=graphs.n_node)
segment_max_expanded = e3nn.scatter_max(nodes, map_back=True, nel=graphs.n_node)
print(segment_max_expanded)
# segment_max_expanded = jnp.asarray([segment_max[0], segment_max[0], segment_max[1], segment_max[2]])

expected = 0 + jnp.log(1 + e3nn.scatter_sum(jnp.exp(nodes - 0), nel=graphs.n_node))
computed = segment_max + jnp.log(jnp.exp(-segment_max) + e3nn.scatter_sum(jnp.exp(nodes - segment_max_expanded), nel=graphs.n_node))

expected, computed

In [ ]:
import analysis


In [ ]:
analysis.load_from_workdir("/Users/ameyad/Documents/spherical-harmonic-net/potato_workdirs/workdirs/mace/interactions=1/l=0/channels=32")

In [ ]:
lmax = 3
irreps = e3nn.Irreps(e3nn.Irrep.iterator(lmax))
e3nn.IrrepsArray(irreps=irreps, array=jnp.ones((10, irreps.dim)))

In [ ]:
e3nn.scatter_sum(data=graphs.nodes, nel=graphs.n_node)